In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import pingouin as pg
from semopy import Model, calc_stats, ModelEffects
import semopy.plot as semplot
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("../../data/intermediate/初步清洗_比赛数据.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

satisfaction_df = df[df.columns[df.columns.str.contains("满意度")]]

time_columns = [
    '自习时间',
    '网络课程时间',
    '实验科研时间',
    '竞赛活动时间',
    '其他学习时间'
]

# 标准化时间变量
scaler = MinMaxScaler()
df[time_columns] = scaler.fit_transform(df[time_columns])
# df
# 初始资源感知（输入）
resource_aware_list = [
    '教室设备满意度',
    '实训室满意度',
    '图书馆满意度',
    '网络资源满意度',
    '体育设施满意度',
    '住宿条件满意度'
]
# 学习投入度隐变量（过程）
learning_engagement_list = [
    '课前预学',
    '课堂参与',
    '课后复习',
    '延伸阅读',
    '自习时间',
    # '网络课程时间',
    '实验科研时间',
    '竞赛活动时间',
    # '其他学习时间'
]
# 师生关系（过程）
teacher_student_relationship_list = [
    '师德师风满意度',
    '关爱学生满意度',
    '教学投入满意度',
    '班主任工作满意度',
    '学业指导满意度'
]
# 专业课体验（过程）
major_course_experience_list = [
    '专业课知识融合',
    '专业课解决问题能力',
    '专业课交叉融合',
    '专业课实践结合',
    '专业课努力程度',
    '专业课前沿内容'
]
# 体美劳课体验（过程）
phy_art_labour_course_experience_list = [
    '体育教育满意度',
    '美育教育满意度',
    '劳动教育满意度'
]
# 综合满意度（输出）
overall_satisfaction_list = [
    '学校整体满意度',
    '思政课总体满意度',
    '实习内容满意度',
    '教师总体满意度'
]
# 计算平均数
df['初始资源感知'] = df[resource_aware_list].mean(axis=1)
df['学习投入度'] = df[learning_engagement_list].mean(axis=1)
df['师生关系'] = df[teacher_student_relationship_list].mean(axis=1)
df['专业课体验'] = df[major_course_experience_list].mean(axis=1)
df['综合满意度'] = df[overall_satisfaction_list].mean(axis=1)
df['体美劳体验'] = df[phy_art_labour_course_experience_list].mean(axis=1)
latent_variable_definitions = {
    '初始资源感知': resource_aware_list,
    '学习投入度': learning_engagement_list, # 注意这里用的是组合后的列表
    '师生关系': teacher_student_relationship_list,
    '专业课体验': major_course_experience_list,
    '体美劳体验': phy_art_labour_course_experience_list,
    '综合满意度': overall_satisfaction_list
}

for latent_var, indicators in latent_variable_definitions.items():
    try:
        # 从df中提取这些指标列，并删除含有NaN的行
        data_subset = df[indicators].dropna()

        if len(indicators) > 1 and len(data_subset) > 0:
            # 计算Cronbach's Alpha
            alpha_result = pg.cronbach_alpha(data_subset)
            alpha_value = alpha_result[0]
            ci_lower, ci_upper = alpha_result[1]  # 解包置信区间

            # 解释信度水平
            if alpha_value >= 0.9:
                reliability_level = "优秀"
            elif alpha_value >= 0.8:
                reliability_level = "良好"
            elif alpha_value >= 0.7:
                reliability_level = "可接受"
            elif alpha_value >= 0.6:
                reliability_level = "勉强接受"
            else:
                reliability_level = "不可接受"

            print(f"{latent_var}:")
            print(f"  Alpha值: {alpha_value:.3f} ({reliability_level})")
            print(f"  95%置信区间: [{ci_lower:.3f}, {ci_upper:.3f}]")
            print(f"  指标数量: {len(indicators)}")
            print(f"  有效样本数: {len(data_subset)}")
            print()

        else:
            print(f"{latent_var}: 指标数量不足或无有效数据")
            print()

    except Exception as e:
        print(f"计算 {latent_var} 的信度时出错: {e}")
        print()

print("\n=== 潜变量描述性统计 ===")
latent_vars = ['初始资源感知', '学习投入度', '师生关系', '专业课体验', '综合满意度']
print(df[latent_vars].describe().round(3))

In [ ]:
model_desc = """
# 测量模型
初始资源感知 =~ 教室设备满意度 + 实训室满意度 + 图书馆满意度 + 网络资源满意度 + 体育设施满意度
学习投入度 =~ 课前预学 + 课堂参与 + 课后复习 + 延伸阅读 + 自习时间 + 实验科研时间 + 竞赛活动时间
师生关系 =~ 师德师风满意度 + 关爱学生满意度 + 教学投入满意度
专业课体验 =~ 专业课知识融合 + 专业课解决问题能力 + 专业课交叉融合 + 专业课实践结合 + 专业课努力程度 + 专业课前沿内容
体美劳体验 =~ 体育教育满意度 + 美育教育满意度 + 劳动教育满意度
综合满意度 =~ 学校整体满意度 + 思政课总体满意度 + 实习内容满意度 + 教师总体满意度

# 结构模型
学习投入度 ~ 初始资源感知 + 师生关系
专业课体验 ~ 初始资源感知 + 师生关系
体美劳体验 ~ 初始资源感知 + 师生关系
综合满意度 ~ 学习投入度 + 专业课体验
综合满意度 ~ 初始资源感知

# 潜变量协方差
师生关系 ~~ 初始资源感知
学习投入度 ~~ 专业课体验

# 残差相关（根据变量逻辑）
# 自习时间 ~~ 网络课程时间
图书馆满意度 ~~ 网络资源满意度
教室设备满意度 ~~ 实训室满意度
自习时间 ~~ 实验科研时间
# 竞赛活动时间 ~~ 其他学习时间
专业课知识融合 ~~ 专业课解决问题能力
自习时间 ~~ 实验科研时间
# 竞赛活动时间 ~~ 其他学习时间
专业课知识融合 ~~ 专业课解决问题能力
"""
model = Model(model_desc)

# 拟合数据（使用你已有的df）
result = model.fit(df)

# 查看结果
stats = calc_stats(model)
print("\n=== 模型描述统计 ===")
print(stats)
# 查看标准化路径系数
inspection = model.inspect(std_est=True)
print("\n=== 标准化路径系数 ===")
print(inspection)
